# Bulk Muscle Growth Predictor - GUI + Flask API

This notebook contains:
1. **Trained ML model** for predicting muscle growth
2. **Flask API server** (`/bulk`, `/health`)
3. **Interactive HTML GUI** with real-time predictions over 3, 6, 9, 12 months
4. **Responsive design** with form validation and visual feedback

---

In [1]:
# Install required packages
# %pip install pandas numpy scikit-learn matplotlib seaborn joblib flask flask-cors

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os
from typing import List, Dict

In [3]:
# Load the data
try:
    df = pd.read_csv('mostCommonExercises.csv')
    print("Dataset loaded successfully!")
except Exception as e:
    print(f"Error loading dataset: {e}")
    exit()

print("Dataset Overview:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nTarget variable distribution:")
print(df['muscle_size_increase_cm2'].describe())

Dataset loaded successfully!
Dataset Overview:
         exercise_name target_muscle_group      type  age gender  sets  reps  \
0  Barbell Bench Press               Chest  Compound   32      M     4    10   
1  Barbell Bench Press               Chest  Compound   28      F     3    12   
2  Barbell Bench Press               Chest  Compound   39      M     5     8   
3  Barbell Bench Press               Chest  Compound   35      F     4    10   
4  Barbell Bench Press               Chest  Compound   26      M     3    12   

   weight  frequency  protein  calories  sleep    experience  \
0      80          3      150      2800    7.5  Intermediate   
1      45          2      130      2600    7.2      Beginner   
2      90          4      170      3100    8.0      Advanced   
3      50          3      135      2700    7.4  Intermediate   
4      85          2      145      2750    7.6  Intermediate   

   muscle_size_increase_cm2  genetic_advantage  actual_fat_loss  \
0                   

In [4]:
# Baseline growth model
def get_baseline_params(age, gender, experience, current_size_cm, workout_time_years):
    if gender == 'M':
        base_limit = 45
        age_factor = max(0.7, 1 - (age - 25) * 0.01)
    else:
        base_limit = 30
        age_factor = max(0.7, 1 - (age - 25) * 0.008)
    
    M_max = base_limit * age_factor
    remaining_potential = max(0, M_max - (current_size_cm * 0.2))
    
    k_base = {'Beginner': 0.20, 'Intermediate': 0.10, 'Advanced': 0.05}
    k = k_base.get(experience, 0.10) * (1 / (1 + workout_time_years * 0.1))
    
    return remaining_potential, k

def calculate_baseline_growth(row, time_months, current_size_cm, workout_time_years):
    M_max, k = get_baseline_params(row['age'], row['gender'], row['experience'], current_size_cm, workout_time_years)
    M0 = current_size_cm * 0.2
    baseline_growth = (M_max - M0) * (1 - np.exp(-k * time_months))
    baseline_cm2 = baseline_growth * 5
    return max(baseline_cm2, 0.1)

In [5]:
df['baseline_growth'] = df.apply(
    lambda row: calculate_baseline_growth(row, time_months=3, current_size_cm=0, workout_time_years=0), axis=1
)
df['adjustment_factor'] = df['muscle_size_increase_cm2'] / df['baseline_growth']
df['adjustment_factor'] = np.clip(df['adjustment_factor'], 0.01, 10.0)

print("\nBaseline vs Actual Growth:")
print(f"Mean baseline growth: {df['baseline_growth'].mean():.2f} cm2")
print(f"Mean actual growth: {df['muscle_size_increase_cm2'].mean():.2f} cm2")
print(f"Mean adjustment factor: {df['adjustment_factor'].mean():.2f}")


Baseline vs Actual Growth:
Mean baseline growth: 47.45 cm2
Mean actual growth: 1.88 cm2
Mean adjustment factor: 0.05


In [6]:
# Feature Engineering
le_gender = LabelEncoder()
le_exercise = LabelEncoder()
le_muscle_group = LabelEncoder()
le_category = LabelEncoder()
le_experience = LabelEncoder()

df['gender_encoded'] = le_gender.fit_transform(df['gender'])
df['exercise_name_encoded'] = le_exercise.fit_transform(df['exercise_name'])
df['muscle_group_encoded'] = le_muscle_group.fit_transform(df['target_muscle_group'])
df['category_encoded'] = le_category.fit_transform(df['type'])
df['experience_encoded'] = le_experience.fit_transform(df['experience'])

df['protein_per_kg'] = df['protein'] / np.maximum(df['current_weight'] * 0.45, 1)
df['volume'] = df['sets'] * df['reps']
df['intensity'] = df['weight'] / np.maximum(df['reps'], 1)
df['calories_per_kg'] = df['calories'] / np.maximum(df['current_weight'] * 0.45, 1)

In [7]:
# Train Model
features = [
    'age', 'gender_encoded', 'exercise_name_encoded', 'sets', 'reps', 'weight',
    'frequency', 'protein', 'calories', 'sleep', 'experience_encoded',
    'muscle_group_encoded', 'category_encoded', 'protein_per_kg', 'volume',
    'intensity', 'calories_per_kg', 'genetic_advantage'
]

X = df[features].replace([np.inf, -np.inf], np.nan).fillna(df[features].median())
y = df['adjustment_factor']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Performance:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Model Performance:
Mean Squared Error: 0.0000
R² Score: 0.9824


In [8]:
# Save model and encoders
os.makedirs('model', exist_ok=True)
joblib.dump(rf_model, 'model/bulkPredictor.pkl')
joblib.dump(le_gender, 'model/le_gender.pkl')
joblib.dump(le_exercise, 'model/le_exercise.pkl')
joblib.dump(le_experience, 'model/le_experience.pkl')
joblib.dump(le_muscle_group, 'model/le_muscle_group.pkl')
joblib.dump(le_category, 'model/le_category.pkl')

['model/le_category.pkl']

---
# Flask API + GUI

Below is a **complete Flask app** with an **interactive HTML GUI**.

In [9]:
%%writefile app.py
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
import joblib
import numpy as np
import pandas as pd
import os
from typing import List, Dict

app = Flask(__name__)
CORS(app)

# Load model and encoders
model_path = 'model/bulkPredictor.pkl'
encoders = {
    'gender': joblib.load('model/le_gender.pkl'),
    'exercise': joblib.load('model/le_exercise.pkl'),
    'muscle_group': joblib.load('model/le_muscle_group.pkl'),
    'category': joblib.load('model/le_category.pkl'),
    'experience': joblib.load('model/le_experience.pkl')
}
model = joblib.load(model_path)

def get_baseline_params(age, gender, experience, current_size_cm, workout_time_years):
    if gender == 'M':
        base_limit = 45
        age_factor = max(0.7, 1 - (age - 25) * 0.01)
    else:
        base_limit = 30
        age_factor = max(0.7, 1 - (age - 25) * 0.008)
    M_max = base_limit * age_factor
    remaining_potential = max(0, M_max - (current_size_cm * 0.2))
    k_base = {'Beginner': 0.20, 'Intermediate': 0.10, 'Advanced': 0.05}
    k = k_base.get(experience, 0.10) * (1 / (1 + workout_time_years * 0.1))
    return remaining_potential, k

def calculate_baseline_growth(age, gender, experience, time_months, current_size_cm, workout_time_years):
    M_max, k = get_baseline_params(age, gender, experience, current_size_cm, workout_time_years)
    M0 = current_size_cm * 0.2
    baseline_growth = (M_max - M0) * (1 - np.exp(-k * time_months))
    return max(baseline_growth * 5, 0.1)

def safe_transform(encoder, value, name="value"):
    try:
        if value in encoder.classes_:
            return encoder.transform([value])[0]
        print(f"{name} '{value}' not in training data")
        return 0
    except:
        return 0

@app.route('/')
def home():
    exercises = sorted(encoders['exercise'].classes_)
    html = f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Bulk Predictor</title>
        <style>
            :root {{ --primary: #2563eb; --success: #10b981; --danger: #ef4444; }}
            body {{ font-family: 'Segoe UI', sans-serif; background: #f8fafc; margin: 0; padding: 20px; }}
            .container {{ max-width: 1000px; margin: auto; background: white; border-radius: 12px; box-shadow: 0 10px 25px rgba(0,0,0,0.1); overflow: hidden; }}
            header {{ background: var(--primary); color: white; padding: 1.5rem; text-align: center; }}
            .form-section {{ padding: 2rem; }}
            .input-group {{ margin-bottom: 1rem; }}
            label {{ display: block; margin-bottom: 0.5rem; font-weight: 600; color: #374151; }}
            input, select {{ width: 100%; padding: 0.75rem; border: 1px solid #d1d5db; border-radius: 6px; font-size: 1rem; }}
            .exercise-item {{ border: 1px solid #e5e7eb; border-radius: 8px; padding: 1rem; margin-bottom: 1rem; background: #f9fafb; }}
            .btn {{ background: var(--primary); color: white; border: none; padding: 0.75rem 1.5rem; border-radius: 6px; cursor: pointer; font-size: 1rem; }}
            .btn:hover {{ background: #1d4ed8; }}
            .add-btn {{ background: var(--success); margin-bottom: 1rem; }}
            .remove-btn {{ background: var(--danger); float: right; padding: 0.25rem 0.5rem; font-size: 0.8rem; }}
            .results {{ margin-top: 2rem; padding: 1.5rem; background: #f0f9ff; border-radius: 8px; }}
            .timeline {{ display: flex; gap: 1rem; margin-top: 1rem; }}
            .card {{ flex: 1; background: white; padding: 1rem; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
            .loading {{ text-align: center; padding: 2rem; color: #6b7280; }}
            .error {{ color: var(--danger); margin-top: 1rem; }}
        </style>
    </head>
    <body>
        <div class="container">
            <header>
                <h1>Bulk Muscle Growth Predictor</h1>
                <p>Predict muscle gains over 3, 6, 9, and 12 months</p>
            </header>
            <div class="form-section">
                <form id="predictForm">
                    <div class="input-group">
                        <label>Age</label>
                        <input type="number" id="age" min="18" max="100" value="25" required>
                    </div>
                    <div class="input-group">
                        <label>Gender</label>
                        <select id="gender" required>
                            <option value="M">Male</option>
                            <option value="F">Female</option>
                        </select>
                    </div>
                    <div class="input-group">
                        <label>Experience Level</label>
                        <select id="experience" required>
                            <option>Beginner</option>
                            <option>Intermediate</option>
                            <option selected>Advanced</option>
                        </select>
                    </div>
                    <div class="input-group">
                        <label>Weekly Training Frequency (days)</label>
                        <input type="number" id="frequency" min="1" max="7" value="4" required>
                    </div>
                    <div class="input-group">
                        <label>Daily Protein (g)</label>
                        <input type="number" id="protein" min="50" max="300" value="150" required>
                    </div>
                    <div class="input-group">
                        <label>Daily Calories</label>
                        <input type="number" id="calories" min="1500" max="5000" value="2800" required>
                    </div>
                    <div class="input-group">
                        <label>Average Sleep (hours)</label>
                        <input type="number" step="0.1" id="sleep" min="5" max="12" value="7.5" required>
                    </div>
                    <div class="input-group">
                        <label>Current Muscle Size (cm², optional)</label>
                        <input type="number" step="0.1" id="current_size" value="0">
                    </div>
                    <div class="input-group">
                        <label>Years Training (optional)</label>
                        <input type="number" step="0.1" id="workout_years" value="0">
                    </div>

                    <h3>Exercises</h3>
                    <button type="button" class="btn add-btn" onclick="addExercise()">+ Add Exercise</button>
                    <div id="exercises-container">
                        <div class="exercise-item">
                            <select class="exercise-select" required>
                                <option value="">Select Exercise</option>
                                {''.join([f'<option>{ex}</option>' for ex in exercises])}
                            </select>
                            <input type="number" placeholder="Sets" class="sets" min="1" max="10" required>
                            <input type="number" placeholder="Reps" class="reps" min="1" max="20" required>
                            <input type="number" placeholder="Weight (kg)" class="weight" min="1" max="300" required>
                            <button type="button" class="btn remove-btn" onclick="this.parentElement.remove()">×</button>
                        </div>
                    </div>

                    <button type="submit" class="btn" style="width:100%; margin-top:1.5rem; font-size:1.1rem;">
                        Predict Growth
                    </button>
                </form>

                <div id="loading" class="loading" style="display:none;">Predicting...</div>
                <div id="error" class="error" style="display:none;"></div>
                <div id="results" class="results" style="display:none;"></div>
            </div>
        </div>

        <script>
            function addExercise() {{
                const container = document.getElementById('exercises-container');
                const div = document.createElement('div');
                div.className = 'exercise-item';
                div.innerHTML = `
                    <select class="exercise-select" required>
                        <option value="">Select Exercise</option>
                        {''.join([f'<option>{ex}</option>' for ex in exercises])}
                    </select>
                    <input type="number" placeholder="Sets" class="sets" min="1" max="10" required>
                    <input type="number" placeholder="Reps" class="reps" min="1" max="20" required>
                    <input type="number" placeholder="Weight (kg)" class="weight" min="1" max="300" required>
                    <button type="button" class="btn remove-btn" onclick="this.parentElement.remove()">×</button>
                `;
                container.appendChild(div);
            }}

            document.getElementById('predictForm').onsubmit = async (e) => {{
                e.preventDefault();
                document.getElementById('loading').style.display = 'block';
                document.getElementById('results').style.display = 'none';
                document.getElementById('error').style.display = 'none';

                const exercises = Array.from(document.querySelectorAll('.exercise-item')).map(item => ({{
                    exercise_name: item.querySelector('.exercise-select').value,
                    sets: parseInt(item.querySelector('.sets').value),
                    reps: parseInt(item.querySelector('.reps').value),
                    weight: parseInt(item.querySelector('.weight').value),
                    target_muscle_group: '',
                    exercise_category: 'Compound'
                }}));

                if (exercises.some(ex => !ex.exercise_name)) {{
                    showError('Please select an exercise for each entry.');
                    return;
                }}

                const data = {{
                    age: parseInt(document.getElementById('age').value),
                    gender: document.getElementById('gender').value,
                    experience: document.getElementById('experience').value,
                    frequency: parseInt(document.getElementById('frequency').value),
                    protein: parseFloat(document.getElementById('protein').value),
                    calories: parseInt(document.getElementById('calories').value),
                    sleep: parseFloat(document.getElementById('sleep').value),
                    current_size_cm: parseFloat(document.getElementById('current_size').value) || 0,
                    workout_time_years: parseFloat(document.getElementById('workout_years').value) || 0,
                    exercises: exercises
                }};

                try {{
                    const res = await fetch('/bulk', {{
                        method: 'POST',
                        headers: {{ 'Content-Type': 'application/json' }},
                        body: JSON.stringify(data)
                    }});
                    const result = await res.json();
                    if (res.ok) {{
                        displayResults(result.result);
                    }} else {{
                        showError(result.error || 'Prediction failed');
                    }}
                }} catch (err) {{
                    showError('Network error: ' + err.message);
                }}
                document.getElementById('loading').style.display = 'none';
            }};

            function showError(msg) {{
                document.getElementById('loading').style.display = 'none';
                const el = document.getElementById('error');
                el.textContent = msg;
                el.style.display = 'block';
            }}

            function displayResults(data) {{
                const container = document.getElementById('results');
                container.innerHTML = '<h3>Predicted Muscle Growth (cm²)</h3><div class="timeline">' +
                    Object.entries(data).map(([months, preds]) => `
                        <div class="card">
                            <h4>${{months}} Months</h4>
                            ${{Object.entries(preds).map(([k, v]) => `<div><strong>${{k}}</strong>: ${{v}}</div>`).join('')}}
                        </div>
                    `).join('') + '</div>';
                container.style.display = 'block';
            }}
        </script>
    </body>
    </html>
    '''
    return render_template_string(html)

@app.route('/bulk', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        intervals = [3, 6, 9, 12]
        result = {}

        for interval in intervals:
            predictions = predict_growth_for_interval(data, interval)
            result[str(interval)] = predictions

        return jsonify({'result': result})
    except Exception as e:
        return jsonify({'error': str(e)}), 400

def predict_growth_for_interval(data, time_months):
    age = data['age']
    gender = data['gender']
    experience = data['experience']
    current_size_cm = data.get('current_size_cm', 0)
    workout_time_years = data.get('workout_time_years', 0)

    muscle_groups = {}
    for ex in data['exercises']:
        name = ex['exercise_name']
        # Map exercise to muscle group (simplified)
        mg_map = {
            'Chest': ['Bench', 'Press', 'Flyes', 'Dips', 'Push-ups'],
            'Back': ['Rows', 'Pulldowns', 'Pull-ups', 'Deadlift'],
            'Arms': ['Curls', '21s']
        }
        muscle = 'Unknown'
        for m, keywords in mg_map.items():
            if any(k in name for k in keywords):
                muscle = m
                break
        if muscle not in muscle_groups:
            muscle_groups[muscle] = []
        muscle_groups[muscle].append(ex)

    predictions = {}
    for muscle, ex_list in muscle_groups.items():
        total_volume = sum(ex['sets'] * ex['reps'] * ex['weight'] for ex in ex_list)
        avg_sets = np.mean([ex['sets'] for ex in ex_list])
        avg_reps = np.mean([ex['reps'] for ex in ex_list])
        equiv_weight = total_volume / (avg_sets * avg_reps) if avg_sets and avg_reps else 50

        baseline = calculate_baseline_growth(age, gender, experience, time_months, current_size_cm, workout_time_years)

        feature_row = {
            'age': age,
            'gender_encoded': safe_transform(encoders['gender'], gender),
            'exercise_name_encoded': safe_transform(encoders['exercise'], ex_list[0]['exercise_name']),
            'sets': avg_sets,
            'reps': avg_reps,
            'weight': equiv_weight,
            'frequency': data['frequency'],
            'protein': data['protein'],
            'calories': data['calories'],
            'sleep': data['sleep'],
            'experience_encoded': safe_transform(encoders['experience'], experience),
            'muscle_group_encoded': 0,
            'category_encoded': 0,
            'protein_per_kg': data['protein'] / 70,
            'volume': avg_sets * avg_reps,
            'intensity': equiv_weight / avg_reps,
            'calories_per_kg': data['calories'] / 70,
            'genetic_advantage': 3
        }

        X = pd.DataFrame([feature_row])
        X = X.reindex(columns=model.feature_names_in_, fill_value=0)
        adj_factor = model.predict(X)[0]
        growth = baseline * adj_factor
        predictions[f"{muscle} growth"] = f"{growth:.2f} cm2"

    return predictions

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy'})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=3001, debug=False)


Overwriting app.py


---
# Run the Server

```bash
python app.py
```

Open in browser: [http://localhost:3001](http://localhost:3001)

---

**Features:**
- Add/remove exercises
- Real-time predictions
- 3/6/9/12 month forecasts
- Mobile-friendly
- Form validation
- Error handling